# SFcalculator_torch testing for my purposes

## With experimental data

In [2]:
from SFC_Torch.Fmodel import SFcalculator

In [3]:
from SFC_Torch.utils import try_gpu
from SFC_Torch.io import PDBParser

In [4]:
import reciprocalspaceship as rs
import numpy as np
import gemmi

In [5]:
coeff = rs.read_cif("tests/resources/4yuo-sf.cif")
structure = gemmi.make_structure_from_block(
    gemmi.cif.read("tests/resources/4yuo.cif")[0]
)
parsed_structure = PDBParser(structure)
sfcalculator = SFcalculator(
    parsed_structure, mtzdata=coeff, set_experiment=True, device=try_gpu()
)

In [6]:
sfcalculator.inspect_data(verbose=True)

Solvent Percentage: tensor(0.4067, device='cuda:0')
Grid size: [108, 144, 240]


In [7]:
Fprotein = sfcalculator.calc_fprotein(Return=True)
print(Fprotein)

tensor([-2178.9512-1.4324e-04j,  1969.9944+3.7103e-04j,   556.3026-3.4717e-05j,
         ...,    14.1725+1.2387e-04j,   -26.2972+3.8349e+00j,
           -7.1229-1.6533e+01j], device='cuda:0')


In [8]:
Fsolvent = sfcalculator.calc_fsolvent(Return=True, dmin_mask=6.0, dmin_nonzero=3.0)
print(Fsolvent)

tensor([ 33875.3984-0.0054j, -80130.7969-0.0127j, -18305.1797-0.0018j,
         ...,      0.0000+0.0000j,      0.0000+0.0000j,
             0.0000+0.0000j], device='cuda:0')


## Try without experimental data

In [9]:
sfcalculator = SFcalculator(
    "tests/resources/4yuo.pdb", set_experiment=False, device=try_gpu(), dmin=1.2
)

In [10]:
sfcalculator.inspect_data(verbose=True)

Solvent Percentage: tensor(0.4067, device='cuda:0')
Grid size: [108, 144, 240]


In [11]:
Fprotein = sfcalculator.calc_fprotein(Return=True)
print(Fprotein)

tensor([-6.7019e+03-5.8627e-04j,  2.5258e+03+4.6574e-04j,
         7.1024e+02+2.0864e-04j,  ...,
         9.2197e+00+5.1824e+00j, -1.0886e+01-3.1965e+00j,
         3.6690e-02-9.3127e+00j], device='cuda:0')


In [12]:
Fsolvent = sfcalculator.calc_fsolvent(Return=True, dmin_mask=6.0, dmin_nonzero=3.0)
print(Fsolvent)

tensor([ 305408.1250+0.0170j, -106013.8828-0.0233j,  -35555.8555-0.0308j,
         ...,       0.0000+0.0000j,       0.0000+0.0000j,
              0.0000+0.0000j], device='cuda:0')


In [13]:
sfcalculator._set_scales(False)
Ftotal = sfcalculator.calc_ftotal(Return=True)
print(Ftotal)

tensor([ 1.0018e+05+5.3519e-03j, -3.4565e+04-7.6916e-03j,
        -1.1724e+04-1.0549e-02j,  ...,
         8.0442e+00+4.5217e+00j, -9.4964e+00-2.7883e+00j,
         3.1996e-02-8.1214e+00j], device='cuda:0')


## Test with intermediate Chroma outputs
not exactly the intermediate Chroma output, but should be the same shape, etc. [1, residues, 4, 3]

In [14]:
import torch
from chroma import Protein
from einops import rearrange

/home/kchrispens/chroma/chroma/layers/graph.py:27: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [15]:
from chroma.layers.structure.mvn import BackboneMVNGlobular

mvn = BackboneMVNGlobular(covariance_model="globular")
multiply_corr = mvn._multiply_R

In [16]:
protein = Protein("tests/resources/4yuo.cif")
protein = protein.to_XCS(device=try_gpu(), all_atom=False)
print(protein[0].size())
bb_protein = Protein.from_XCS(*protein)
bb_protein.to_PDB("tests/resources/4yuo_bb.pdb")
# X = torch.randn(*protein[0].size(), device=try_gpu()).float()
# print(X.size())

torch.Size([1, 165, 4, 3])


In [17]:
# X = rearrange(X, "b r a c -> (b r a) c")
# print(X.size()) # (N, 3)

In [18]:
# just force it to be the right size, not sure where the 323 is coming from since there are only 163 residues
X = torch.randn(1, 652, 3, device=try_gpu()).float()
X.requires_grad = True

In [19]:
structure = gemmi.read_pdb("tests/resources/4yuo_bb.pdb")
structure.spacegroup_hm = "P 21 21 21"
print(structure.spacegroup_hm)
structure.cell = gemmi.UnitCell(42.9, 52.43, 89.11, 90, 90, 90)
print(structure.cell)

P 21 21 21
<gemmi.UnitCell(42.9, 52.43, 89.11, 90, 90, 90)>


In [20]:
sfcalculator = SFcalculator(
    PDBParser(structure), set_experiment=False, device=try_gpu(), dmin=1.2
)

In [21]:
print(sfcalculator.n_atoms)
fmodel = sfcalculator.calc_fprotein_batch(X, Return=True).squeeze()
print(fmodel)
fmodel_no_gauss = sfcalculator.calc_fprotein(Return=True)
print(torch.all(fmodel == fmodel_no_gauss))

652
tensor([17396.4727+1.5154e-03j, 16802.3164+2.9449e-03j, 15855.7910+4.2648e-03j,
         ...,    30.3296-4.3893e+00j,  -198.9884-3.2138e+01j,
           43.6444+2.1310e+00j], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor(False, device='cuda:0')


## Test loss

In [22]:
loss = -torch.linalg.norm(fmodel - fmodel_no_gauss) ** 2
print(loss)

tensor(-4.4969e+10, device='cuda:0', grad_fn=<NegBackward0>)


In [23]:
grad = torch.autograd.grad(loss, X)
print(grad)

(tensor([[[ 2.1910e+07, -1.2764e+08, -3.1818e+07],
         [-5.7520e+07, -6.0786e+07, -3.6444e+06],
         [ 3.5443e+07,  5.7573e+07, -8.0529e+07],
         ...,
         [-8.9247e+07, -7.6644e+07,  9.8883e+06],
         [-6.8079e+07, -4.3642e+07, -2.7826e+07],
         [ 1.1243e+08,  1.1209e+08, -9.5021e+06]]], device='cuda:0'),)


In [24]:
print(grad[0].size())

torch.Size([1, 652, 3])


## Test if batch and normal produce the same results

In [30]:
X, _, _ = Protein("tests/resources/4yuo_bb.pdb").to_XCS(device=try_gpu(), all_atom=False)
X = rearrange(X, "b r a c -> b (r a) c")
X.requires_grad = True
print(X.size())

torch.Size([1, 652, 3])


In [31]:
fmodel_from_pdb = sfcalculator.calc_fprotein_batch(X, Return=True).squeeze()
print(fmodel_from_pdb)

tensor([-3551.6123+0.0000e+00j,   678.0986+6.1035e-05j,   301.7066+6.1035e-05j,
         ...,   -28.3438+1.1367e+02j,    22.0244-7.8736e+01j,
           54.1495-1.8673e+02j], device='cuda:0', grad_fn=<SqueezeBackward0>)


In [32]:
print(fmodel_no_gauss)

tensor([-3551.6123-2.7881e-04j,   678.0987+6.0769e-05j,   301.7065+6.8289e-05j,
         ...,   -28.3438+1.1367e+02j,    22.0243-7.8736e+01j,
           54.1495-1.8673e+02j], device='cuda:0')


In [33]:
loss = -torch.linalg.norm(fmodel_from_pdb - fmodel_no_gauss) ** 2
print(loss)
grad = torch.autograd.grad(loss, X)
print(grad)

tensor(-4.9836e-05, device='cuda:0', grad_fn=<NegBackward0>)
(tensor([[[ 0.0816,  0.1471, -0.0970],
         [ 0.0742,  0.0038,  0.1265],
         [-0.1284, -0.1098,  0.0813],
         ...,
         [ 0.0184,  0.0988,  0.2238],
         [-0.1325, -0.1024,  0.0212],
         [-0.1692, -0.0654, -0.2295]]], device='cuda:0'),)


In [34]:
print(torch.max(grad[0]))

tensor(0.4907, device='cuda:0')


## optimize over variables

In [8]:
import torch
import time

In [9]:
from SFC_Torch.utils import r_factor

In [10]:
kall = torch.tensor(1.0, device=try_gpu(), requires_grad=True)
kaniso = torch.normal(0.001, 0.001, size=[6], device=try_gpu(), requires_grad=True)
ksol = torch.tensor(0.3, device=try_gpu(), requires_grad=True)
bsol = torch.tensor(15.0, device=try_gpu(), requires_grad=True)

In [11]:
params = [kall, kaniso, ksol, bsol]
optimizer = torch.optim.Adam(params, lr=0.05)

In [12]:
def scale_train(optimizer, sfcalculator, n_steps=1000, loss_track=[], verbose=True):
    def scale_steptrain(optimizer, sfcalculator):
        Fmodel = sfcalculator.Calc_Ftotal(
            kall=kall, kaniso=kaniso, ksol=ksol, bsol=bsol
        )
        Fmodel_mag = torch.abs(Fmodel)
        loss = torch.sum((sfcalculator.Fo - Fmodel_mag) ** 2)
        r_work, r_free = r_factor(
            sfcalculator.Fo, Fmodel_mag, sfcalculator.rwork_id, sfcalculator.rfree_id
        )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        return loss, r_work, r_free

    for k in range(n_steps):
        start_time = time.time()

        temp = scale_steptrain(optimizer, sfcalculator)
        loss_track.append([i.detach().cpu().numpy() for i in temp])
        time_this_round = round(time.time() - start_time, 3)
        str_ = "Time: " + str(time_this_round)

        if verbose:
            print("Scale", *loss_track[-1], str_, flush=True)
    return loss_track

In [13]:
loss_track = scale_train(
    optimizer, sfcalculator, n_steps=2000, loss_track=[], verbose=True
)

Scale 2360606981.272027 0.7934486742107228 1.2524560150122919 Time: 0.424
Scale 1381669763.1487303 0.7505973969193179 1.086307705440703 Time: 0.004
Scale 745217182.6176398 0.7401504120141958 0.9802809824964632 Time: 0.004
Scale 362849179.21097815 0.7188504971233238 0.8779463719591027 Time: 0.004
Scale 157667109.15107003 0.6972303428533211 0.7888401735149541 Time: 0.004
Scale 67806394.38946718 0.6857418766046256 0.7246369192021016 Time: 0.004
Scale 45642531.281120464 0.6893363763971943 0.6917977046216831 Time: 0.004
Scale 50355407.980374895 0.7094794602447297 0.7213077594798187 Time: 0.004
Scale 72853717.31063293 0.7429923313018277 0.7782158420085415 Time: 0.004
Scale 103590399.96907102 0.7694520640983036 0.8299605255222956 Time: 0.004
Scale 134951625.75334418 0.7883953769433174 0.8705908178241503 Time: 0.004
Scale 162197377.34005517 0.802053579852674 0.89980126956626 Time: 0.004
Scale 182279747.53382033 0.810872699075718 0.9191345379314891 Time: 0.004
Scale 193897469.1653086 0.81559168

In [30]:
print(kall)
print(kaniso)
print(ksol)
print(bsol)

tensor(0.9361, device='cuda:0', requires_grad=True)
tensor([-0.0170, -0.0076, -0.0127, -1.2057, -0.2516, -0.7643], device='cuda:0',
       requires_grad=True)
tensor(0.0177, device='cuda:0', requires_grad=True)
tensor(18.1952, device='cuda:0', requires_grad=True)


### Replace the mask part

In [34]:
sfcalculator.Fmask_HKL = torch.tensor(
    Fmask_complex, device=try_gpu(), dtype=torch.complex64
)

In [40]:
kall = torch.tensor(1.0, device=try_gpu(), requires_grad=True)
kaniso = torch.normal(0.001, 0.001, size=[6], device=try_gpu(), requires_grad=True)
ksol = torch.tensor(3.0, device=try_gpu(), requires_grad=True)
bsol = torch.tensor(15.0, device=try_gpu(), requires_grad=True)

In [41]:
params = [kall, kaniso, ksol, bsol]
optimizer = torch.optim.Adam(params, lr=0.05)

In [42]:
loss_track = scale_train(
    optimizer, sfcalculator, n_steps=2000, loss_track=[], verbose=True
)

Scale 376047457.55528164 0.37064829711832836 0.5426720035513318 Time: 0.011
Scale 322419604.9018515 0.48036122583938085 0.6207319612150738 Time: 0.009
Scale 278943418.4443972 0.5437075297479375 0.6684698743842548 Time: 0.009
Scale 240644075.89388227 0.5555942853422928 0.6697069739450926 Time: 0.009
Scale 206400826.60149938 0.5369381457340847 0.6411978609916129 Time: 0.009
Scale 175881950.50546926 0.4975517352426858 0.5933420529960486 Time: 0.011
Scale 149313789.14508086 0.45907900866861184 0.5468217638400155 Time: 0.009
Scale 127620289.8877142 0.44078411565694886 0.5232754355033543 Time: 0.009
Scale 109410628.37800242 0.4251103966098057 0.5009415373004613 Time: 0.009
Scale 92697745.10059658 0.39602448067920476 0.4629052486695722 Time: 0.009
Scale 79167916.56571051 0.37977954247507895 0.4381849025734263 Time: 0.009
Scale 69497566.41826531 0.3966003296427725 0.4490252023676877 Time: 0.009
Scale 63573015.15061243 0.44311069605448883 0.48835171913869674 Time: 0.009
Scale 60424108.410885915

In [43]:
print(kall)
print(kaniso)
print(ksol)
print(bsol)

tensor(0.9547, device='cuda:0', requires_grad=True)
tensor([-0.0134, -0.0052, -0.0098, -1.2479, -0.3192, -0.7457], device='cuda:0',
       requires_grad=True)
tensor(0.4036, device='cuda:0', requires_grad=True)
tensor(18.8924, device='cuda:0', requires_grad=True)


### Replace the protein part

In [51]:
sfcalculator.Fprotein_HKL = torch.tensor(
    Fcalc_complex, device=try_gpu(), dtype=torch.complex64
)

In [52]:
kall = torch.tensor(1.0, device=try_gpu(), requires_grad=True)
kaniso = torch.normal(0.001, 0.001, size=[6], device=try_gpu(), requires_grad=True)
ksol = torch.tensor(0.3, device=try_gpu(), requires_grad=True)
bsol = torch.tensor(15.0, device=try_gpu(), requires_grad=True)

In [53]:
params = [kall, kaniso, ksol, bsol]
optimizer = torch.optim.Adam(params, lr=0.05)

In [54]:
loss_track = scale_train(
    optimizer, sfcalculator, n_steps=2000, loss_track=[], verbose=True
)

Scale 2362679129.936879 0.7703731343888437 1.2386004382413518 Time: 0.011
Scale 1382787890.751635 0.7373640643693977 1.0759429894669335 Time: 0.009
Scale 745738409.422499 0.7324282219742781 0.9730798536035633 Time: 0.009
Scale 362935717.878269 0.7130820064667867 0.8734116272410787 Time: 0.009
Scale 157377258.09491217 0.6923293143764615 0.78478306967064 Time: 0.009
Scale 67264711.56535427 0.6814828499020628 0.7214018156614841 Time: 0.009
Scale 44973217.67225708 0.685728590430035 0.6890739705430816 Time: 0.008
Scale 49607678.578197196 0.7060947367071306 0.7183484918093542 Time: 0.009
Scale 72011914.71763179 0.739672608363055 0.7755141629463813 Time: 0.009
Scale 102682543.94318643 0.7661323616215401 0.8275242972419219 Time: 0.008
Scale 134061035.46630746 0.7851364135587567 0.8681127672683074 Time: 0.008
Scale 161337547.90838543 0.798811481389618 0.8974872744957532 Time: 0.009
Scale 181477974.05964866 0.8075826864840802 0.9167873630273163 Time: 0.012
Scale 193159817.4718702 0.8121613499337

In [15]:
print(kall)
print(kaniso)
print(ksol)
print(bsol)

tensor(0.9361, device='cuda:0', requires_grad=True)
tensor([-0.0170, -0.0076, -0.0127, -1.2062, -0.2515, -0.7642], device='cuda:0',
       requires_grad=True)
tensor(0.0177, device='cuda:0', requires_grad=True)
tensor(18.1913, device='cuda:0', requires_grad=True)
